In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scrapedData//'
outputDir = '..//summarizedData//'
dataset = 'rivals'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dicti
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
rivalsData = mergeSourceFiles(dataset, inputDir, sourceFiles)

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    if (playerString is not None):
        return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state") 
    else:
        return 'ERROR'

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
createNewID(idConfig[dataset], rivalsData, '_')
df = pandas.DataFrame(rivalsData)
df.head()

In [ ]:
##With this - since I only want to keep 4 columns I'm going to create a new list of dicts
finalRivals = []
for record in rivalsData:
    finalRecord = {}
    finalRecord['ID'] = record['ID']
    finalRecord['rivals_stars'] = record['stars']
    finalRecord['rivals_natlRank'] = record['nationalRank']
    finalRecord['rivals_posRank'] = record['positionRank']
    finalRecord['rivals_stateRank'] = record['stateRank']
    finalRivals.append(finalRecord)
print(finalRivals)

In [ ]:
with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(finalRivals))